In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

In [ ]:
!pip install pystruct

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'
SAVE_PATH = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/Lab'

In [ ]:
# Search for all image files in testing set...
all_images = glob(os.path.join(DATA_PATH,'Videos_and_stills/TestingSet/Lab/*/*/*/Images/Quad*/*.jpg'))
len(all_images)

In [ ]:
# meta-parameters
'''
@param blockSize Size of a pixel neighborhood 
       that is used to calculate a threshold value for the pixel.
@param C Constant subtracted from the mean or weighted mean 
       (see the details below). Normally, it is positive but may be zero 
       or negative as well.
@param k_size morphology structuring element size
'''
blockSize  = 301#1
C_constant = 2
k_size     = 11

min_area = 40000
max_area = 300000

bottom_cut = 150
horiz_cut = 200

In [ ]:
k_25 = np.ones((25, 25), np.uint8)
k_120 = np.ones((120, 120), np.uint8)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k_size, k_size))

for i in range(len(all_images)):
#i = 0
    im   = cv2.imread(all_images[i])
    rgb  = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    th1  = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize, C_constant)

    #kernel = np.ones((k_size, k_size), np.uint8)
    erosion = cv2.erode(th1, kernel, iterations=2)
    close = cv2.dilate(erosion, kernel, iterations=1)

    '''
    @param mode cv2.RETR_EXTERNAL retrieves only the extreme outer contours.
    @param method cv2.CHAIN_APPROX_SIMPLE compresses horizontal, vertical, 
           and diagonal segments and leaves only their end points. For example, 
           an up-right rectangular contour is encoded with 4 points.
    '''
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        area = cv2.contourArea(c)
        if area > min_area: #and area < max_area:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(rgb, (x, y), (x + w, y + h), (36, 255, 12), 5)
            close[y:y+h, x:x+w] = 0

    fig, axes = plt.subplots(1, 2, figsize=(16, 12))
    #axes[0].imshow(rgb)

    close[:, :horiz_cut] = 0
    close[:, close.shape[1] - horiz_cut:] = 0
    close[close.shape[0] - bottom_cut:, :] = 0
    close = cv2.dilate(close, kernel, iterations=1)
    
    # to remove leftover dots
    t = cv2.erode(close, k_25, iterations=1)
    mask = cv2.dilate(t, k_120, iterations=1)

    axes[0].imshow(close)
    axes[1].imshow(close & mask)
    for i in range(len(axes.flat)):
        axes.flat[i].axis('off')
    plt.show() #pause(0.1)

In [ ]:
mask_file = os.path.join(SAVE_PATH, all_images[i].split('/')[-1].split('.')[0] + '_mask.png')
jpeg_file = os.path.join(SAVE_PATH, all_images[i].split('/')[-1])

In [ ]:
cv2.imwrite(mask_file, close & mask)

In [ ]:
cv2.imwrite(jpeg_file, im)

In [ ]:
good = close.copy()

In [ ]:
k_size = 25
kernel = np.ones((k_size, k_size), np.uint8)
t = cv2.erode(close, kernel, iterations=1)

k_size = 120
kernel = np.ones((k_size, k_size), np.uint8)
mask = cv2.dilate(t, kernel, iterations=1)

In [ ]:
plt.imshow()

In [ ]:
#plt.imshow(erosion)

In [ ]:
close.shape

In [ ]:
mask

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 12))
axes[0].imshow(close)
#clean_mask = close[mask == 1] = 1
axes[1].imshow(mask)
for i in range(len(axes.flat)):
    axes.flat[i].axis('off')
plt.show() #pause(0.1)

In [ ]:
import numpy as np
try:
    import cPickle as pickle
except ImportError:
    import pickle

from pystruct import learners
import pystruct.models as crfs
from pystruct.utils import SaveLogger

In [ ]:
!pip install cvxopt

In [ ]:
#data_train = pickle.load()
# https://rebeccabilbro.github.io/convert-py2-pickles-to-py3/
with open('/scratch/ssd/data/CRF_Tut/data_train.pickle', 'rb') as f:
    data_train = pickle.load(f, encoding='latin1')

In [ ]:
#data_train = pickle.load(open("/scratch/ssd/data/CRF_Tut/data_train.pickle"))
C = 0.01

n_states = 21
print("number of samples: %s" % len(data_train['X']))
class_weights = 1. / np.bincount(np.hstack(data_train['Y']))
class_weights *= 21. / np.sum(class_weights)
print(class_weights)

model = crfs.EdgeFeatureGraphCRF(inference_method='qpbo',
                                 class_weight=class_weights,
                                 symmetric_edge_features=[0, 1],
                                 antisymmetric_edge_features=[2])

experiment_name = "edge_features_one_slack_trainval_%f" % C

ssvm = learners.NSlackSSVM(
    model, verbose=2, C=C, max_iter=100000, n_jobs=-1,
    tol=0.0001, show_loss_every=5,
    logger=SaveLogger(experiment_name + ".pickle", save_every=100),
    inactive_threshold=1e-3, inactive_window=10, batch_size=100)
ssvm.fit(data_train['X'], data_train['Y'])

data_val = pickle.load(open("data_val_dict.pickle"))
y_pred = ssvm.predict(data_val['X'])

# we throw away void superpixels and flatten everything
y_pred, y_true = np.hstack(y_pred), np.hstack(data_val['Y'])
y_pred = y_pred[y_true != 255]
y_true = y_true[y_true != 255]

print("Score on validation set: %f" % np.mean(y_true == y_pred))

In [ ]:
!pip install pyqpbo